# Trabalho Prático 3
**Grupo 22**

Alexis Correia - A102495 <br>
João Fonseca - A102512 <br>


## Problema 3

Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $\,\mathsf{A5/1}$ e o FOTS usando BitVec’s que aì foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados

$$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad$$ 
$$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$
                
1. Codifique em “z3”  o SFOTS assim definido.
2. Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.

## Resolução

Definidos o conjunto dos estados, o estado inicial e a condição de erro, falta definir as transições deste SFOTS. <bl>
Neste caso, as transições são: 
$$
\begin{array}{c}
()
\\\vee\\
()
\end{array}
$$

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import BVType

n0, n1, n2 = 19, 22, 23
cB0, cB1, cB2 = 8, 10, 10
s0 = BV("1110010000000000000", n0)
s1 = BV("1100000000000000000000", n1)
s2 = BV("11100000000000010000000", n2)

def declare(s,i):
    state = {}
    state['x0'] = Symbol('x0'+'!'+s+str(i), BVType(n0))
    state['x1'] = Symbol('x1'+'!'+s+str(i), BVType(n1))
    state['x2'] = Symbol('x2'+'!'+s+str(i), BVType(n2))
    return state

def init(state):
    A = BVSGT(state['x0'], BV(0,n0))
    B = BVSGT(state['x1'], BV(0,n1))
    C = BVSGT(state['x2'], BV(0,n2))
    return And(A,B,C)

def cBit(state):
    c0 = BVExtract(state['lfsr0'], cB0, cB0)
    c1 = BVExtract(state['lfsr1'], cB1, cB1)
    c2 = BVExtract(state['lfsr2'], cB2, cB2)
    if ((c0 & c1) | (c1 & c2) | (c0 & c2)):
        r = BV(1,1)
    else:
        r = BV(0,1)
    return r

def trans(curr,prox): #####
    c = cBit(curr)
    t0 = And(Equals(BVExtract(curr['lfsr0'],cB0, cB0),c),
             Equals(prox['lfsr0'], BVXor(BVLShl(curr['lfsr0'],1),BVXor(curr['lfsr0'],s0))))
    t1 = And(Equals(BVExtract(curr['lfsr1'],cB1, cB1),c),
             Equals(prox['lfsr1'], BVXor(BVLShl(curr['lfsr1'],1),BVXor(curr['lfsr1'],s1))))
    t2 = And(Equals(BVExtract(curr['lfsr2'],cB2, cB2),c),
             Equals(prox['lfsr2'], BVXor(BVLShl(curr['lfsr2'],1),BVXor(curr['lfsr2'],s2))))
    
    return Or(And(t0,t1), And(t0,t2), And(t1,t2), And(t0,t1,t2))

def error(state):
    return Not(init(state))

def genTrace(init,trans,error,n): #vars = ['x0','x1','x2']
    with Solver(name="z3") as s:
        X = [declare('X',i) for i in range(n+1)] 
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        E = [error(X[i]) for i in range(n)]
        if s.solve([I,And(Tks),Not(And(E))]):
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))
        else:
            print("ERRO")


In [ ]:
def invert(trans):
    return lambda curr, nxt: trans(nxt, curr)

In [ ]:
''' 
def check_property(self, prop):
        """Property Directed Reachability approach without optimizations."""
        print("Checking property %s..." % prop)

        while True:
            cube = self.get_bad_state(prop)
            if cube is not None:
                # Blocking phase of a bad state
                if self.recursive_block(cube):
                    print("--> Bug found at step %d" % (len(self.frames)))
                    break
                else:
                    print("   [PDR] Cube blocked '%s'" % str(cube))
            else:
                # Checking if the last two frames are equivalent i.e., are inductive
                if self.inductive():
                    print("--> The system is safe!")
                    break
                else:
                    print("   [PDR] Adding frame %d..." % (len(self.frames)))
                    self.frames.append(TRUE())
'''